# 📊 01 - Análise Exploratória de Dados
Este notebook apresenta uma visão inicial do conjunto de dados utilizado no projeto de previsão de inadimplência.


- Coleta dos Dados
- Visão Geral do Conjunto de Dados

In [1]:
#Importação de pacotes
import warnings
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import numpy as np
import joblib # Utilizado para salvar o modelo preditivo
from sklearn.preprocessing import LabelEncoder #Utilizada para fazer o OneHotEncoding
from sklearn.metrics import mean_squared_error,precision_score, recall_score, f1_score, accuracy_score, roc_auc_score, confusion_matrix
from imblearn import under_sampling, over_sampling #Utilizada para fazer o balanceamento de dados
from imblearn.over_sampling import SMOTE #Utilizada para fazer o balanceamento de dados
from sklearn.preprocessing import MinMaxScaler #Utilizada para fazer a padronização dos dados
from sklearn.metrics import r2_score # Utilizado para medir a acuracia do modelo preditivo
import pyodbc as sql # Substituição de pymssql para conexão SQL

warnings.filterwarnings("ignore") 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
#pd.options.display.float_format = '{:.2f}'.format

In [2]:
#Importação do arquivo de dados

# Função de conexão com o SQL Server 
def connect_to_sql_server_modelos():
    conn = sql.connect(
        'DRIVER={ODBC Driver 17 for SQL Server};'  # Usando ODBC Driver 17 para SQL Server
        'SERVER=localhost;'  # Substitua 'localhost' pelo endereço do seu servidor
        'DATABASE=MODELOS_PREDITIVOS;'  # Nome do banco de dados
        'UID=usuario_python;'  # Nome do usuário
        'PWD=123456;'  # Senha
    )
    return conn

# Conectando ao banco de dados
conn = connect_to_sql_server_modelos()

# Realizando a consulta e carregando os dados em um DataFrame
df_original = pd.read_sql_query('SELECT * FROM EXTRACAO_DADOS_SISTEMA', conn)

# Exibindo as primeiras linhas do DataFrame
print(df_original.head())

# Fechando a conexão
conn.close()

   NUMERO_CONTRATO DATA_ASSINATURA_CONTRATO TIPO_FINANCIAMENTO  TAXA_AO_ANO  \
0            41637               2012-03-23        IMOBILIARIO         15.0   
1            67958               2012-01-24        IMOBILIARIO         15.0   
2            68502               2014-05-23        IMOBILIARIO         18.0   
3            68731               2015-10-23        IMOBILIARIO         15.0   
4            68996               2017-01-24        IMOBILIARIO         17.0   

   PZ_FINANCIAMENTO CIDADE_CLIENTE ESTADO_CLIENTE  RENDA_MENSAL_CLIENTE  \
0               180      FORTALEZA             CE                4800.0   
1               200      SAO PAULO             SP                6800.0   
2               200      SAO PAULO             SP                6800.0   
3               200      SAO PAULO             SP                6800.0   
4               200      SAO PAULO             SP                6800.0   

   QT_PC_ATRASO  QT_DIAS_PRIM_PC_ATRASO  QT_TOTAL_PC_PAGAS  VL_TOTAL_PC_PA

### Analise Exploratória - Visão Geral
<br>
Aqui o objetivo desta analise é ter uma visão geral do conjunto de dados, compreendendo seu tamanho, variaveis, 
tipo de dados e período dos dados coletados.

In [3]:
#Tamanho do conjunto de dados. 
df_original.shape

(10415, 20)

In [4]:
#Visão geral do conjunto de dados
df_original.head(5)

,NUMERO_CONTRATO,DATA_ASSINATURA_CONTRATO,TIPO_FINANCIAMENTO,TAXA_AO_ANO,PZ_FINANCIAMENTO,CIDADE_CLIENTE,ESTADO_CLIENTE,RENDA_MENSAL_CLIENTE,QT_PC_ATRASO,QT_DIAS_PRIM_PC_ATRASO,QT_TOTAL_PC_PAGAS,VL_TOTAL_PC_PAGAS,QT_PC_PAGA_EM_DIA,QT_DIAS_MIN_ATRASO,QT_DIAS_MAX_ATRASO,QT_DIAS_MEDIA_ATRASO,VALOR_FINANCIAMENTO,VALOR_PARCELA,IDADE_DATA_ASSINATURA_CONTRATO,INADIMPLENTE_COBRANCA
0,41637,2012-03-23,IMOBILIARIO,15.0,180,FORTALEZA,CE,4800.0,0,0,97,120078.00,72,3,506,152,73767.0,471.29,70.0,SIM
1,67958,2012-01-24,IMOBILIARIO,15.0,200,SAO PAULO,SP,6800.0,0,0,103,156759.47,42,1,349,93,323251.0,1858.69,28.0,SIM
2,68502,2014-05-23,IMOBILIARIO,18.0,200,SAO PAULO,SP,6800.0,0,0,98,277125.05,83,1,151,39,529419.0,3123.57,60.0,SIM
3,68731,2015-10-23,IMOBILIARIO,15.0,200,SAO PAULO,SP,6800.0,0,0,87,351395.19,66,1,3797,206,499753.0,2873.58,41.0,SIM
4,68996,2017-01-24,IMOBILIARIO,17.0,200,SAO PAULO,SP,6800.0,0,0,71,344249.83,57,1,36,10,722057.0,4224.03,63.0,SIM


In [5]:
# Analisando os dados missing podemos constatar o seguinte:
# Variavel DADE_DATA_ASSINATURA_CONTRATO possui 1 registro
# Variavel VL_TOTAL_PC_PAGAS possui 4 registros

# Como temos um total de 10.415 observações então excluir 5 observações NÃO afetará nosso trabalho

df_original.isnull().sum()

NUMERO_CONTRATO                   0
DATA_ASSINATURA_CONTRATO          0
TIPO_FINANCIAMENTO                0
TAXA_AO_ANO                       0
PZ_FINANCIAMENTO                  0
CIDADE_CLIENTE                    0
ESTADO_CLIENTE                    0
RENDA_MENSAL_CLIENTE              0
QT_PC_ATRASO                      0
QT_DIAS_PRIM_PC_ATRASO            0
QT_TOTAL_PC_PAGAS                 0
VL_TOTAL_PC_PAGAS                 4
QT_PC_PAGA_EM_DIA                 0
QT_DIAS_MIN_ATRASO                0
QT_DIAS_MAX_ATRASO                0
QT_DIAS_MEDIA_ATRASO              0
VALOR_FINANCIAMENTO               0
VALOR_PARCELA                     0
IDADE_DATA_ASSINATURA_CONTRATO    1
INADIMPLENTE_COBRANCA             0
dtype: int64

In [6]:
#Informações básicas sobre tipos de variáveis
df_original.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10415 entries, 0 to 10414
Data columns (total 20 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   NUMERO_CONTRATO                 10415 non-null  int64  
 1   DATA_ASSINATURA_CONTRATO        10415 non-null  object 
 2   TIPO_FINANCIAMENTO              10415 non-null  object 
 3   TAXA_AO_ANO                     10415 non-null  float64
 4   PZ_FINANCIAMENTO                10415 non-null  int64  
 5   CIDADE_CLIENTE                  10415 non-null  object 
 6   ESTADO_CLIENTE                  10415 non-null  object 
 7   RENDA_MENSAL_CLIENTE            10415 non-null  float64
 8   QT_PC_ATRASO                    10415 non-null  int64  
 9   QT_DIAS_PRIM_PC_ATRASO          10415 non-null  int64  
 10  QT_TOTAL_PC_PAGAS               10415 non-null  int64  
 11  VL_TOTAL_PC_PAGAS               10411 non-null  float64
 12  QT_PC_PAGA_EM_DIA               

In [7]:
# Avaliar o período dos dados coletados
inicio = pd.to_datetime(df_original['DATA_ASSINATURA_CONTRATO']).dt.date.min()
fim = pd.to_datetime(df_original['DATA_ASSINATURA_CONTRATO']).dt.date.max()
print('Período dos dados - De:', inicio, 'Até:',fim)

Período dos dados - De: 2012-01-10 Até: 2022-11-14


In [8]:
# Total de valores únicos de cada variável
# A variável TIPO_FINANCIAMENTO possuI valor unico, então será retirada do nosso DataFrame

valores_unicos = []
for i in df_original.columns[0:20].tolist():
    print(i, ':', len(df_original[i].astype(str).value_counts()))
    valores_unicos.append(len(df_original[i].astype(str).value_counts()))

NUMERO_CONTRATO : 10415
DATA_ASSINATURA_CONTRATO : 1070
TIPO_FINANCIAMENTO : 1
TAXA_AO_ANO : 111
PZ_FINANCIAMENTO : 9
CIDADE_CLIENTE : 525
ESTADO_CLIENTE : 27
RENDA_MENSAL_CLIENTE : 7
QT_PC_ATRASO : 130
QT_DIAS_PRIM_PC_ATRASO : 227
QT_TOTAL_PC_PAGAS : 137
VL_TOTAL_PC_PAGAS : 9861
QT_PC_PAGA_EM_DIA : 129
QT_DIAS_MIN_ATRASO : 122
QT_DIAS_MAX_ATRASO : 534
QT_DIAS_MEDIA_ATRASO : 323
VALOR_FINANCIAMENTO : 834
VALOR_PARCELA : 1826
IDADE_DATA_ASSINATURA_CONTRATO : 79
INADIMPLENTE_COBRANCA : 2


In [9]:
# exportando os dados para um arquivo excel
df_original.to_excel('C:/Users/leojo/Desktop/PROJETO_INADIMPLENCIA_COMPLETO/data\processed/df_original_pt_1.xlsx', index=False)
